# Sierra Leone EDA Notebook
# Objective: Profile, clean, and explore Sierra Leone's solar dataset end-to-end with metric-specific outlier detection.


In [ ]:
# Sierra Leone EDA - Setup and Imports
import os
import sys
import warnings
from typing import List

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path

# Add parent directories to path for utility imports
sys.path.append(str(Path(__file__).resolve().parents[2]))
from app.utils import (
    detect_outliers_ghi,
    detect_outliers_dni,
    detect_outliers_dhi,
    detect_solar_metric_outliers,
)

warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")
sns.set_context("talk")

COUNTRY = "sierraleone"
DATA_DIR = Path("../data").resolve()
RAW_DATA_PATH = DATA_DIR / "sierraleone-bumbuna.csv"
CLEAN_DATA_PATH = DATA_DIR / "sierraleone_clean.csv"

# Columns of interest
NUMERIC_CANDIDATES: List[str] = [
    "GHI", "DNI", "DHI", "Tamb", "TModA", "TModB",
    "ModA", "ModB", "WS", "WSgust", "WD", "RH", "BP"
]
TIME_COLUMN_CANDIDATES: List[str] = ["Timestamp", "timestamp", "time", "Date", "Datetime"]
CLEANING_FLAG_CANDIDATES: List[str] = ["Cleaning", "cleaned", "is_cleaned"]

def find_first_column(df: pd.DataFrame, candidates: List[str]) -> str | None:
    for c in candidates:
        if c in df.columns:
            return c
    return None

def existing_columns(df: pd.DataFrame, cols: List[str]) -> List[str]:
    return [c for c in cols if c in df.columns]

print(f"Expecting raw CSV at: {RAW_DATA_PATH}")
if not RAW_DATA_PATH.exists():
    print("WARNING: Raw data file not found. Place the CSV at:", RAW_DATA_PATH)


In [ ]:
# Load data
try:
    df = pd.read_csv(RAW_DATA_PATH)
    print("Loaded:", df.shape)
    # Try to parse a timestamp column if present
    time_col = find_first_column(df, TIME_COLUMN_CANDIDATES)
    if time_col is not None:
        df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
        df = df.sort_values(by=time_col)
    else:
        print("No time-like column found in:", TIME_COLUMN_CANDIDATES)
except FileNotFoundError:
    df = pd.DataFrame()
    print("Data not found. Proceed to place the CSV and re-run this cell.")


In [ ]:
# Basic profile: head, info, dtypes
if not df.empty:
    display(df.head())
    display(df.tail())
    print("\nData types:\n", df.dtypes)
    print("\nShape:", df.shape)
else:
    print("DataFrame is empty. Load data first.")


In [ ]:
# Summary statistics & missing-value report
if not df.empty:
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    display(df[num_cols].describe().T)

    na_series = df.isna().sum().sort_values(ascending=False)
    na_pct = (na_series / len(df) * 100).round(2)
    missing_report = pd.DataFrame({"missing": na_series, "%": na_pct})
    display(missing_report[missing_report["%"] > 0])

    gt5 = missing_report[missing_report["%"] > 5]
    if not gt5.empty:
        print("Columns with >5% nulls:")
        display(gt5)
else:
    print("DataFrame is empty. Load data first.")



In [ ]:
# Metric-specific outlier detection for GHI, DNI, DHI
if not df.empty:
    print("="*60)
    print("METRIC-SPECIFIC OUTLIER DETECTION")
    print("="*60)
    
    # Apply metric-specific outlier detection
    outlier_flags, combined_mask = detect_solar_metric_outliers(df)
    
    # Report findings for each metric
    for metric in ["GHI", "DNI", "DHI"]:
        if metric in df.columns:
            outlier_col = f"{metric}_outlier"
            if outlier_col in outlier_flags.columns:
                outlier_count = outlier_flags[outlier_col].sum()
                outlier_pct = (outlier_count / len(df) * 100).round(2)
                print(f"\n{metric}:")
                print(f"  Outliers detected: {outlier_count} ({outlier_pct}%)")
                
                # Show statistics for outliers vs non-outliers
                if outlier_count > 0:
                    normal_values = df.loc[~outlier_flags[outlier_col], metric]
                    outlier_values = df.loc[outlier_flags[outlier_col], metric]
                    print(f"  Normal range: {normal_values.min():.1f} - {normal_values.max():.1f} W/m²")
                    print(f"  Outlier range: {outlier_values.min():.1f} - {outlier_values.max():.1f} W/m²")
    
    print(f"\nTotal rows with any outlier: {combined_mask.sum()} ({combined_mask.sum()/len(df)*100:.2f}%)")
    
    # Add outlier flags to dataframe for visualization
    for col in outlier_flags.columns:
        df[col] = outlier_flags[col]
else:
    print("DataFrame is empty. Load data first.")


In [ ]:
# Create cleaned dataset with median imputation
if not df.empty:
    cleaned = df.copy()
    key_cols = existing_columns(cleaned, ["GHI", "DNI", "DHI", "ModA", "ModB", "WS", "WSgust"])
    for c in key_cols:
        if cleaned[c].isna().any():
            median_val = cleaned[c].median()
            cleaned[c] = cleaned[c].fillna(median_val)
            print(f"Imputed {c}: {cleaned[c].isna().sum()} → 0 missing (median: {median_val:.2f})")
    
    display(cleaned[key_cols].describe().T)
    
    # Store for export
    cleaned_df = cleaned
else:
    print("DataFrame is empty. Load data first.")


In [ ]:
# Export cleaned dataset
def export_clean(df_original: pd.DataFrame) -> None:
    if df_original.empty:
        print("Empty DataFrame; skipping export.")
        return
    cleaned = df_original.copy()
    key_cols = existing_columns(cleaned, ["GHI", "DNI", "DHI", "ModA", "ModB", "WS", "WSgust"])
    for c in key_cols:
        cleaned[c] = cleaned[c].fillna(cleaned[c].median())

    os.makedirs(os.path.dirname(CLEAN_DATA_PATH), exist_ok=True)
    cleaned.to_csv(CLEAN_DATA_PATH, index=False)
    print("Exported cleaned CSV to:", CLEAN_DATA_PATH)

if 'cleaned_df' in globals() and not cleaned_df.empty:
    export_clean(cleaned_df)
else:
    print("No cleaned data to export.")


In [ ]:
# Time series analysis: GHI, DNI, DHI, Tamb vs time
if not df.empty:
    tcol = find_first_column(df, TIME_COLUMN_CANDIDATES)
    if tcol is not None and pd.api.types.is_datetime64_any_dtype(df[tcol]):
        ts_cols = existing_columns(df, ["GHI", "DNI", "DHI", "Tamb"])
        if ts_cols:
            fig, axes = plt.subplots(len(ts_cols), 1, figsize=(14, 3.2*len(ts_cols)), sharex=True)
            if len(ts_cols) == 1:
                axes = [axes]
            for ax, c in zip(axes, ts_cols):
                ax.plot(df[tcol], df[c], label=c, alpha=0.7)
                ax.set_ylabel(c)
                ax.legend(loc="upper right")
            axes[-1].set_xlabel(tcol)
            plt.suptitle("Time Series: core variables - Sierra Leone")
            plt.tight_layout()
            plt.show()
        else:
            print("No TS columns among:", ["GHI", "DNI", "DHI", "Tamb"])    
    else:
        print("Time column not found or not datetime; skipping TS plots.")
else:
    print("DataFrame is empty. Load data first.")


In [ ]:
# Distribution boxplots for GHI, DNI, DHI
if not df.empty:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=False)
    for ax, metric in zip(axes, ["GHI", "DNI", "DHI"]):
        if metric in df.columns:
            sns.boxplot(data=df, y=metric, ax=ax, color="#60a5fa")
            ax.set_title(f"{metric} distribution - Sierra Leone")
            ax.set_ylabel(f"{metric} (W/m²)")
    plt.tight_layout()
    plt.show()
else:
    print("DataFrame is empty. Load data first.")


In [ ]:
# Correlation heatmap
if not df.empty:
    corr_cols = existing_columns(df, ["GHI", "DNI", "DHI", "TModA", "TModB", "Tamb", "WS", "WSgust", "RH"])
    if corr_cols:
        corr = df[corr_cols].corr()
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True, center=0)
        plt.title("Correlation Heatmap - Sierra Leone")
        plt.tight_layout()
        plt.show()
    else:
        print("No numeric columns for correlation heatmap.")
else:
    print("DataFrame is empty. Load data first.")


In [ ]:
# Scatter plots: WS, WSgust, WD vs GHI; RH vs Tamb & GHI
if not df.empty:
    pairs = [
        ("WS", "GHI"), ("WSgust", "GHI"), ("WD", "GHI"),
        ("RH", "Tamb"), ("RH", "GHI")
    ]
    pairs = [(x, y) for (x, y) in pairs if x in df.columns and y in df.columns]
    if pairs:
        cols = 2
        rows = int(np.ceil(len(pairs) / cols))
        fig, axes = plt.subplots(rows, cols, figsize=(12, 4 * rows))
        axes = np.atleast_1d(axes).ravel()
        for ax, (x, y) in zip(axes, pairs):
            sns.scatterplot(data=df, x=x, y=y, s=10, alpha=0.4, ax=ax)
            sns.regplot(data=df, x=x, y=y, scatter=False, color='red', ax=ax)
            ax.set_title(f"{x} vs {y} - Sierra Leone")
        for ax in axes[len(pairs):]:
            ax.axis("off")
        plt.tight_layout()
        plt.show()
    else:
        print("No valid pairs for scatter plots.")
else:
    print("DataFrame is empty. Load data first.")


In [ ]:
# Histograms for GHI, DNI, DHI
if not df.empty:
    hcols = existing_columns(df, ["GHI", "DNI", "DHI"])
    if hcols:
        fig, axes = plt.subplots(1, len(hcols), figsize=(5*len(hcols), 4))
        if len(hcols) == 1:
            axes = [axes]
        for ax, col in zip(axes, hcols):
            df[col].hist(bins=50, ax=ax, color="#60a5fa", alpha=0.7)
            ax.set_title(f"{col} Histogram - Sierra Leone")
            ax.set_xlabel(f"{col} (W/m²)")
            ax.set_ylabel("Frequency")
        plt.tight_layout()
        plt.show()
    else:
        print("No histogram columns present.")
else:
    print("DataFrame is empty. Load data first.")


In [ ]:
# Cleaning impact: average ModA & ModB by Cleaning flag (if available)
if not df.empty:
    flag_col = find_first_column(df, CLEANING_FLAG_CANDIDATES)
    target_cols = existing_columns(df, ["ModA", "ModB"])
    if flag_col and target_cols:
        grp = df.groupby(flag_col)[target_cols].mean().rename_axis(flag_col)
        display(grp)
        grp.plot(kind="bar", figsize=(8, 4), title="Average ModA/ModB by Cleaning flag - Sierra Leone")
        plt.ylabel("Average")
        plt.tight_layout()
        plt.show()
    else:
        print("Cleaning flag or target cols not present; skipping.")
else:
    print("DataFrame is empty. Load data first.")
